samtools是一个用于操作sam和bam文件（通常是短序列比对工具如bwa,bowtie2,hisat2,tophat2等等产生的）的工具合集，包含有许多命令。以下是常用命令的介绍。

# 1. View

view命令的主要功能是：将sam文件与bam文件互换；然后对bam文件进行各种操作，比如数据的排序(sort)和提取(这些操作是对bam文件进行的，因而当输入为sam文件的时候，不能进行该操作)；最后将排序或提取得到的数据输出为bam或sam（默认的）格式。

bam文件优点：bam文件为二进制文件，占用的磁盘空间比sam文本文件小；利用bam二进制文件的运算速度快。

view命令中，对sam文件头部（序列ID）的输入(-t或-T）和输出(-h)是单独的一些参数来控制的。

```
Usage: samtools view [options] <in.bam>|<in.sam> [region1 [...]]
默认情况下不加 region，则是输出所有的 region.
options:
     -b output BAM
      默认下输出是 SAM 格式文件，该参数设置输出 BAM 格式
     -h print header for the SAM output
      默认下输出的sam 格式文件不带header，该参数设定输出sam文件时带 header 信息
     -H print header only (no alignments)
      仅仅输出文件的头
     -S input is SAM
      默认下输入是 BAM 文件，若是输入是 SAM 文件，则最好加该参数，否则有时候会报错。
     -u uncompressed BAM output (force -b)
      该参数的使用需要有-b参数，能节约时间，但是需要更多磁盘空间。
     -c Instead of printing the alignments, only count them and print the 
      total number. All filter options, such as ‘-f’, ‘-F’ and ‘-q’ , are taken into account.
      过滤功统计功能
     -c print only the count of matching records
     -L FILE  output alignments overlapping the input BED FILE [null]
     -t FILE  list of reference names and lengths (force -S) [null]
      使用一个list文件来作为header的输入
     -T FILE  reference sequence file (force -S) [null]
      使用序列fasta文件作为header的输入
     -o FILE  output file name [stdout]
     -F INT   filtering flag, 0 for unset [0] 
      Skip alignments with bits present in INT [0]
      数字4代表该序列没有比对到参考序列上
      数字8代表该序列的mate序列没有比对到参考序列上
      过滤功能。如F12过滤只有双端map的
     -q INT   minimum mapping quality [0]
        比对的最低质量值，一般认为20就为unique比对了，可以结合上述-bF参数使用使用提取特定的比对结果
```

```
例子：

将sam文件转换成bam文件
samtools view -bS abc.sam > abc.bam
BAM转换为SAM
samtools view -h -o out.sam out.bam 
提取比对到参考序列上的比对结果
samtools view -bF 4 abc.bam > abc.F.bam
提取paired reads中两条reads都比对到参考序列上的比对结果，只需要把两个4+8的值12作为过滤参数即可 
samtools view -bF 12 abc.bam > abc.F12.bam
提取没有比对到参考序列上的比对结果
samtools view -bf 4 abc.bam > abc.f.bam
提取bam文件中比对到caffold1上的比对结果，并保存到sam文件格式
samtools view abc.bam scaffold1 > scaffold1.sam
提取scaffold1上能比对到30k到100k区域的比对结果
samtools view abc.bam scaffold1:30000-100000 > scaffold1_30k-100k.sam
根据fasta文件，将 header 加入到 sam 或 bam 文件中
samtools view -T genome.fasta -h scaffold1.sam > scaffold1.h.sam
```

# 2. Sort

sort对bam文件进行排序。一些软件需要sort的bam或者sam文件，如stringtie，所以必须要sort使用；求depth时，也必须要sort；

```
Usage: samtools sort [-n] [-m <maxMem>] <in.bam> <out.prefix>  
    -m 内存参数默认下是 500,000,000 即500M（不支持K，M，G等缩写）。对于处理大数据时，如果内存够用，则设置大点的值，以节约时间。
    -n 设定排序方式按short reads的ID排序。默认下是按序列在fasta文件中的顺序（即header）和序列从左往右的位点排序。
    
例子：
samtools sort accept.bam accept.sort
最终产生accept.sort.bam
```

# 3. merge

将2个或2个以上的已经sort了的bam文件融合成一个bam文件。融合后的文件已经sort过了的。

```
Usage:   samtools merge [-nr] [-h inh.sam] <out.bam> <in1.bam> <in2.bam>[...]

Options: -n       sort by read names
         -r       attach RG tag (inferred from file names)
         -u       uncompressed BAM output
         -f       overwrite the output BAM if exist
         -1       compress level 1
         -R STR   merge file in the specified region STR [all]
         -h FILE  copy the header in FILE to <out.bam> [in1.bam]
```

# 4. index

必须对bam文件进行默认情况下的排序后，才能进行index。否则会报错。

建立索引后将产生后缀为.bai的文件，用于快速的随机处理。很多情况下需要有bai文件的存在，特别是显示序列比对情况下。     
比如samtool的tview命令就需要；gbrowse2显示reads的比对图形的时候也需要。IGV显示比对情况也需要。    

```
Usage: samtools index <in.bam> [out.index]

例子：
以下两种命令结果一样
samtools index abc.sort.bam
samtools index abc.sort.bam abc.sort.bam.bai
```

# 5. faidx

对fasta文件建立索引,生成的索引文件以.fai后缀结尾。该命令也能依据索引文件快速提取fasta文件中的某一条（子）序列。

```
Usage: samtools faidx <in.bam> [ [...]]

对基因组文件建立索引，方便提取序列。

例子：
samtools faidx genome.fasta

由于有索引文件，可以使用以下命令很快从基因组中提取到fasta格式的子序列
samtools faidx genome.fasta scffold_10 > scaffold_10.fasta
```

# 6. tview

tview能直观的显示出reads比对基因组的情况，和基因组浏览器有点类似。  

需要事先利用利用上面讲的sort和建index命令执行完后，用下述命令。   

```
Usage: samtools tview <aln.bam> [ref.fasta]

当给出参考基因组的时候，会在第一排显示参考基因组的序列，否则，第一排全用N表示。
按下 g ，则提示输入要到达基因组的某一个位点。
例子“scaffold_10:1000"表示到达第10号scaffold的第1000个碱基位点处。
使用H(左）J（上）K（下）L（右）移动显示界面。大写字母移动快，小写字母移动慢。
使用空格建向左快速移动（和 L 类似），使用Backspace键向左快速移动（和 H 类似）。
Ctrl+H 向左移动1kb碱基距离； Ctrl+L 向右移动1kb碱基距离
可以用颜色标注比对质量，碱基质量，核苷酸等。
30～40的碱基质量或比对质量使用白色表示；20～30黄色；10～20绿色；0～10蓝色。
使用点号'.'切换显示碱基和点号；使用r切换显示read name等
还有很多其它的使用说明，具体按 ？ 键来查看。
```

# 7. flagstat

给出BAM文件的比对结果

```
Usage: samtools flagstat <in.bam>

samtools flagstat example.bam
    11945742 + 0 in total (QC-passed reads + QC-failed reads)
    #总共的reads数
    0 + 0 duplicates
    7536364 + 0 mapped (63.09%:-nan%)
    #总体上reads的匹配率
    11945742 + 0 paired in sequencing
    #有多少reads是属于paired reads
    5972871 + 0 read1
    #reads1中的reads数
    5972871 + 0 read2
    #reads2中的reads数
    6412042 + 0 properly paired (53.68%:-nan%)
    #完美匹配的reads数：比对到同一条参考序列，并且两条reads之间的距离符合设置的阈值
    6899708 + 0 with itself and mate mapped
    #paired reads中两条都比对到参考序列上的reads数
    636656 + 0 singletons (5.33%:-nan%)
    #单独一条匹配到参考序列上的reads数，和上一个相加，则是总的匹配上的reads数。
    469868 + 0 with mate mapped to a different chr
    #paired reads中两条分别比对到两条不同的参考序列的reads数
    243047 + 0 with mate mapped to a different chr (mapQ>=5)
    #同上一个，只是其中比对质量>=5的reads的数量
```

# 8. depth

得到每个碱基位点的测序深度,并输出到标准输出，所以要用大于号追加到一个文件。

```
Usage: bam2depth [-r reg] [-q baseQthres] [-Q mapQthres] [-b in.bed] <in1.bam> [...]
    -r 后面跟染色体号（region）
    -q ：计算深度时要求测序碱基质量最低质量值
    -Q ：计算深度时要求比对的最低质量值
    
注意：做depth之前必须做samtools index；

例子
samtools depth accept.bam > depth
```

# 9. 其他命令

```
(1)reheader：替换bam文件的头

samtools reheader <in.header.sam> <in.bam>

(2)idxstats ：统计一个表格，4列，分别为”序列名，序列长度，比对上的reads数，unmapped reads number。
第4列应该是paired reads中有一端能匹配到该scaffold上，而另外一端不匹配到任何scaffolds上的reads数。

samtools idxstats <aln.bam>

(3)rmdup：将由PCR duplicates获得的reads去掉，并只保留最高比对质量的read。

Usage:  samtools rmdup [-sS] 

    -s 对single-end reads。默认情况下，只对paired-end reads
    -S 将Paired-end reads作为single-end reads处理。
```

# 10. 将bam文件转换为fastq文件

```
有时候，我们需要提取出比对到一段参考序列的reads，进行小范围的分析，以利于debug等。
这时需要将bam或sam文件转换为fastq格式。
该网站提供了一个bam转换为fastq的程序：
http://www.hudsonalpha.org/gsl/information/software/bam2fastq

wget http://www.hudsonalpha.org/gsl/static/software/bam2fastq-1.1.0.tgz
tar zxf bam2fastq-1.1.0.tgz
cd bam2fastq-1.1.0
make
./bam2fastq <in.bam>
```

# 11. mpileup

```
samtools还有个非常重要的命令mpileup，以前为pileup。该命令用于生成bcf文件，再使用bcftools进行SNP和Indel的分析。
bcftools是samtool中附带的软件，在samtools的安装文件夹中可以找到。

最常用的参数有2：

 -f 来输入有索引文件的fasta参考序列； 
 -g 输出到bcf格式。用法和最简单的例子如下

Usage: samtools mpileup [-EBug] [-C capQcoef] [-r reg] [-f in.fa] [-l list] [-M capMapQ] [-Q minBaseQ] [-q minMapQ] in.bam [in2.bam [...]]

samtools mpileup -f genome.fasta abc.bam > abc.txt
samtools mpileup -gSDf genome.fasta abc.bam > abc.bcf
samtools mpileup -guSDf genome.fasta abc.bam | bcftools view -cvNg - > abc.vcf

mpileup不使用-u或-g参数时，则不生成二进制的bcf文件，而生成一个文本文件(输出到标准输出)。
该文本文件统计了参考序列中每个碱基位点的比对情况；该文件每一行代表了参考序列中某一个碱基位点的比对结果。
```


# 12. 使用bcftools

```
bcftools和samtools类似，用于处理vcf(variant call format)文件和bcf(binary call format)文件。前者为文本文件，后者为其二进制文件。

bcftools使用简单，最主要的命令是view命令，其次还有index和cat等命令。index和cat命令和samtools中类似。
此处主讲使用view命令来进行SNP和Indel calling。该命令的使用方法和例子为：

bcftools view [-AbFGNQSucgv] [-D seqDict] [-l listLoci] [-s listSample] 
          [-i gapSNPratio] [-t mutRate] [-p varThres] [-P prior] 
          [-1 nGroup1] [-d minFrac] [-U nPerm] [-X permThres] 
          [-T trioType] in.bcf [region]
          
bcftools view -cvNg abc.bcf > snp_indel.vcf

生成的结果文件为vcf格式，有10列，分别是：1 参考序列名；2 varianti所在的left-most位置；3 variant的ID（默认未设置，用’.'表示)；
4 参考序列的allele；5 variant的allele(有多个alleles，则用’,'分隔);6 variant/reference QUALity;7 FILTers applied;
8 variant的信息，使用分号隔开；9 FORMAT of the genotype fields, separated by colon (optional)； 
10 SAMPLE genotypes and per-sample information (optional)。

使用bcftools得到variant calling结果后。需要对结果再次进行过滤。主要依据比对结果中第8列信息。
其中的 DP4 一行尤为重要，提供了4个数据：1 比对结果和正链一致的reads数、2 比对结果和负链一致的reads数、3 比对结果在正链的variant上的reads数、
4 比对结果在负链的variant上的reads数。可以设定 （value3 + value4）大于某一阈值，才算是variant。
```